In [64]:
import torch 

In [ ]:
dispositivo = 'cuda' if torch.cuda.is_available() else 'cpu'

lote = 500
entradas = 50
neuronios = 50

X = torch.rand(lote, entradas, device=dispositivo)

W = torch.rand(neuronios, entradas, device=dispositivo)

B = torch.rand(neuronios, device=dispositivo)

$$y = W\cdot X + b$$
onde:
- $X \in \mathrm{R}^{p \times n}$: Entradas
- $W \in \mathrm{R}^{p \times n}$: Pesos / Parâmetros
- $b \in \mathrm{R}^{p}$: Intercepto / Parâmetros
- $b \in \mathrm{N}^+$: Tamanho do lote (batch)
- $p \in \mathrm{N}^+$: Número de unidades / neurônios

## Implementação sequencial

In [67]:
def linear_sequencial(X, W, B):
    saidas = torch.zeros(lote, neuronios, device=dispositivo)
    for batch in range(lote):
        for neuronio in range(neuronios):
            for entrada in range(entradas):
                saidas[batch, neuronio] += X[batch,entrada] * W[neuronio, entrada]
            saidas[batch, neuronio] += B[neuronio]
    return saidas

linear_sequencial(X,W,B).size()

torch.Size([500, 50])

## Implementação vetorizada com dot product

In [68]:
def linear_vetorizado(X, W, B):
    saidas = torch.zeros(lote, neuronios)
    for batch in range(lote):
        for neuronio in range(neuronios):
            saidas[batch, neuronio] = X[batch] @ W[neuronio] + B[neuronio]
    return saidas

linear_vetorizado(X,W,B).size()

torch.Size([500, 50])

## Implementação vetorizada com Vector Maps

In [69]:
def produto(x,y):
    return x @ y

vmap_produto = torch.func.vmap(produto, in_dims=0)

def neuronio_linear(x,w,b):
    return vmap_produto(x.repeat(neuronios, 1), w) + b

vmap_neuronio = torch.func.vmap(neuronio_linear, in_dims=0)

linear_vmap = lambda X,W,B: vmap_neuronio(X, W.repeat(lote,1,1), B.repeat(lote,1))

linear_vmap(X,W,B).size()

torch.Size([500, 50])